In [1]:
from pprint import pprint
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, f1_score ,recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from multiscorer import MultiScorer
from numpy import average


In [2]:
df= pd.read_csv("Full_features_Regular_Four_classes.csv")
df.columns

Index(['RID', 'DX', 'ABETA', 'AGE', 'APOE4', 'BCCRYING', 'BCDPMOOD',
       'BCENERGY', 'BCINSOMN', 'BMI',
       ...
       'ST32TA', 'ST91CV', 'ST91SA', 'ST91TA', 'ST129CV', 'ST129SA', 'ST129TA',
       'ST130CV', 'ST130SA', 'ST130TA'],
      dtype='object', length=134)

In [3]:
test= pd.read_csv("DX_test_four_classes.csv")
test1=pd.merge(df, test, on='RID', how='inner')
test1=test1.drop('RID',axis = 1 )
y_test = test1.DX                           
X_test= test1.drop('DX',axis = 1 )
for j in range(0,len(test)):
    df=df[df.RID!=test['RID'][j]]

In [4]:
df=df.drop('RID',axis = 1 )
y_train = df.DX                           
X_train= df.drop('DX',axis = 1 )
print(df['DX'].value_counts())

sMCI    425
CN      377
AD      305
pMCI    126
Name: DX, dtype: int64


In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler1 = MinMaxScaler(feature_range=(0, 1))
scaler2 = MinMaxScaler(feature_range=(0, 1))
scaler3 = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler1.fit_transform(X_test)

In [6]:
SFold = StratifiedKFold(n_splits=10, random_state=30, shuffle=True)

In [7]:
# grid serach K Nearest Neighbor
param_grid = {'n_neighbors':[10,20,30,40,50],'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute']}
grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = param_grid, cv = SFold) 
grid_search.fit(X_train, y_train)
print("KNN")
grid_search.best_params_
best_model = grid_search.best_estimator_
best_model

KNN


KNeighborsClassifier(n_neighbors=30, weights='distance')

In [8]:
my_dict_Train={"Accuracy_Train": [],"Precision_Train": [],"Recall_Train": [], "F1_score_Train":[]};
my_dict_STD={"Accuracy_Train": [],"Precision_Train": [],"Recall_Train": [], "F1_score_Train":[]};
my_dict_Test={"Accuracy_Test": [],"Precision_Test": [],"Recall_Test": [], "F1_score_Test":[]};


In [9]:
scorer = MultiScorer({                                               
  'Accuracy_Train': (accuracy_score, {}),
  'Precision_Train': (precision_score, {'average': 'weighted'}) , 
  'Recall_Train': (recall_score, {'average': 'weighted'}),
  'F1_score_Train': (f1_score, {'average': 'weighted'})
})


In [10]:
r=0
for i in range(0,10):
   

    from sklearn.model_selection import StratifiedKFold
    if i==0:
        r=70
    elif i== 1:
        r=80
    elif i== 2:
        r=90
    elif i== 3:
        r=100
    elif i== 4:
        r=130
    elif i==5:
        r=120
    SFold = StratifiedKFold(n_splits=10,  shuffle=True,random_state=r)
    cross_val_score(best_model, X_train, y_train, scoring=scorer, cv=SFold)              

    results = scorer.get_results()                                     

    for metric in results.keys():                                      
        my_dict_Train[metric].append(round(average(results[metric]),2)*100)
        my_dict_STD[metric].append(round(np.std(results[metric]),2)*100)
    y_pred = cross_val_predict(best_model, X_test, y_test, cv=SFold )             
    my_dict_Test["Accuracy_Test"].append(round(100*accuracy_score(y_test,y_pred), 2))
    my_dict_Test["Precision_Test"].append(round(100*precision_score(y_test, y_pred, average='weighted'), 2))
    my_dict_Test["Recall_Test"].append(round(100*recall_score(y_test, y_pred, average='weighted'), 2))
    my_dict_Test["F1_score_Test"].append(round(100*f1_score(y_test, y_pred, average='weighted') , 2))

C:\Users\hager\anaconda3\envs\hager\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hager\anaconda3\envs\hager\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hager\anaconda3\envs\hager\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hager\anaconda3\envs\hager\lib\site-packages\sklearn

In [11]:
for metric in my_dict_Train.keys():
    print("%s: %.3f" % (metric, average(my_dict_Train[metric])))
    print("%s: %.3f" % (metric+"_STD", average(my_dict_STD[metric])))

Accuracy_Train: 70.100
Accuracy_Train_STD: 3.100
Precision_Train: 67.300
Precision_Train_STD: 4.600
Recall_Train: 70.100
Recall_Train_STD: 3.100
F1_score_Train: 67.400
F1_score_Train_STD: 3.100


In [12]:
for metric in my_dict_Test.keys():
    print("%s: %.3f" % (metric, average(my_dict_Test[metric])))

Accuracy_Test: 57.248
Precision_Test: 50.450
Recall_Test: 57.248
F1_score_Test: 52.758
